In [1]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility
from math import sqrt

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/deep-learning-projects/Siamese_Networks/Dataset/PaviaCentre.mat')
arr = mat['pavia']
arr = np.array(arr)
print(arr.shape)

import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/deep-learning-projects/Siamese_Networks/Dataset/PaviaCentre_gt.mat')
arr1 = mat['pavia_gt']
arr1 = np.array(arr1)
print(arr1.shape)

a=[]
label=[]
k=0
for i in range(0,arr1.shape[0]):
    for j in range(0,arr1[i].shape[0]):
        a.append(arr[i][j])
        label.append(arr1[i][j])
        
a=np.array(a)
label=np.array(label)

X_train=[]
y_train=[]
for i in range (0,a.shape[0]):
    if(label[i]==2):
        y_train.append(0)
    if(label[i]==3):
        y_train.append(1)
    if(label[i]==4):
        y_train.append(2)
    if(label[i]==5):
        y_train.append(3)
    if(label[i]==7):
        y_train.append(4)
    if(label[i]==8):
        y_train.append(5)
    if(label[i]==9):
        y_train.append(6)
    if (label[i]==2 or label[i]==3 or label[i]==4 or label[i]==5 or label[i]==7 or label[i]==8 or label[i]==9):
        X_train.append(a[i])
X_train=np.array(X_train)
y_train=np.array(y_train)
print(X_train.shape)
print(y_train.shape)


(1096, 715, 102)
(1096, 715)
(72933, 102)
(72933,)


In [3]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state = 0)

from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit_transform(X_train)
from sklearn.decomposition import PCA
pca = PCA(n_components=64)
X_train = pca.fit_transform(X_train)
print(X_train.shape)

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(72933, 64)


In [4]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/deep-learning-projects/Siamese_Networks/Dataset/PaviaU.mat')
arr = mat['paviaU']
arr = np.array(arr)

import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/deep-learning-projects/Siamese_Networks/Dataset/PaviaU_gt.mat')
arr1 = mat['paviaU_gt']
arr1 = np.array(arr1)
print(arr1.shape)

a=[]
label=[]
k=0
for i in range(0,arr1.shape[0]):
    for j in range(0,arr1[i].shape[0]):
        a.append(arr[i][j])
        label.append(arr1[i][j])
        
a=np.array(a)
label=np.array(label)
print(a.shape)
print(label.shape)

X_train1=[]
y_train1=[]
for i in range (0,a.shape[0]):
    if(label[i]==4):
        y_train1.append(0)
    if(label[i]==1):
        y_train1.append(1)
    if(label[i]==8):
        y_train1.append(2)
    if(label[i]==7):
        y_train1.append(3)
    if(label[i]==9):
        y_train1.append(4)
    if(label[i]==2):
        y_train1.append(5)
    if(label[i]==6):
        y_train1.append(6)
    if (label[i]==4 or label[i]==1 or label[i]==8 or label[i]==7 or label[i]==9 or label[i]==2 or label[i]==6):
        X_train1.append(a[i])
X_train1=np.array(X_train1)
y_train1=np.array(y_train1)

from sklearn.utils import shuffle
X_train1, y_train1 = shuffle(X_train1, y_train1, random_state = 0)
from sklearn.preprocessing import StandardScaler
X_train1 = StandardScaler().fit_transform(X_train1)
from sklearn.decomposition import PCA
pca = PCA(n_components=64)
X_train1 = pca.fit_transform(X_train1)
print(X_train1.shape)


(610, 340)
(207400, 103)
(207400,)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(39332, 64)


In [5]:
print(X_train.max())
print(X_train1.max())
X_train=X_train.astype('float32')
X_train1=X_train1.astype('float32')
X_train=X_train/100
X_train1=X_train1/100

47.423585702771796
104.56701566525479


In [6]:
X_test=X_train[50000:72933,:]
y_test=y_train[50000:72933]
X_train=X_train[0:50000,:]
y_train=y_train[0:50000]

print(X_train.shape)
print(X_train1.shape)
print(X_test.shape)

(50000, 64)
(39332, 64)
(22933, 64)


In [23]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 32 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 64 
num_classes = 7

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_uniform([num_input, num_hidden_1], minval=-4*np.sqrt(6.0/(num_input + num_hidden_1)), maxval=4*np.sqrt(6.0/(num_input + num_hidden_1)))),
    'encoder_h2': tf.Variable(tf.random_uniform([num_hidden_1, num_hidden_2], minval=-4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)), maxval=4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)))),
    'decoder_h1': tf.Variable(tf.random_uniform([num_hidden_2, num_hidden_1], minval=-4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)), maxval=4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)))),
    'decoder_h2': tf.Variable(tf.random_uniform([num_hidden_1, num_input], minval=-4*np.sqrt(6.0/(num_input + num_hidden_1)), maxval=4*np.sqrt(6.0/(num_input + num_hidden_1)))),
    'classifier1_h': tf.Variable(tf.random_uniform([num_hidden_2, 10], minval=-4*np.sqrt(6.0/(10 + num_hidden_2)), maxval=4*np.sqrt(6.0/(10 + num_hidden_2)))),
    'classifier_h': tf.Variable(tf.random_uniform([10, num_classes], minval=-4*np.sqrt(6.0/(10 + num_classes)), maxval=4*np.sqrt(6.0/(10 + num_classes)))),
}
biases = {
    'encoder_b1': tf.Variable(tf.truncated_normal([num_hidden_1])/sqrt(num_hidden_1)),
    'encoder_b2': tf.Variable(tf.truncated_normal([num_hidden_2])/sqrt(num_hidden_2)),
    'decoder_b1': tf.Variable(tf.truncated_normal([num_hidden_1])/sqrt(num_hidden_1)),
    'decoder_b2': tf.Variable(tf.truncated_normal([num_input])/sqrt(num_hidden_2)),
    'classifier1_b': tf.Variable(tf.truncated_normal([10])/sqrt(10)),
    'classifier_b': tf.Variable(tf.truncated_normal([num_classes])/sqrt(num_classes)),
}

In [24]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
classify1 = tf.nn.sigmoid(tf.add(tf.matmul(encoder_op, weights['classifier1_h']), biases['classifier1_b']))
label_pred = tf.nn.softmax(tf.add(tf.matmul(classify1, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [25]:
from keras.utils import np_utils
y_test11 = np_utils.to_categorical(y_test)
y_train11 = np_utils.to_categorical(y_train1)
print(y_train11.shape)
print(y_test11.shape)

(39332, 7)
(22933, 7)


In [26]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [27]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 64
num_batch = 614

# Training
for i in range(0,400):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train1[k:k+batch_size,:]
        batch_y = y_train11[k:k+batch_size,:]
        k += 64
        #print(j)

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))
    print("Epoch:", (i + 1), "accuracy =", "{:.8f}".format(sess.run(accuracy, feed_dict={X: X_train1, Y: y_train11})))

Epoch: 1 cost = 0.02628749
Epoch: 1 accuracy = 0.12786026
Epoch: 2 cost = 0.02511806
Epoch: 2 accuracy = 0.12786026
Epoch: 3 cost = 0.02400085
Epoch: 3 accuracy = 0.12786026
Epoch: 4 cost = 0.02292105
Epoch: 4 accuracy = 0.12786026
Epoch: 5 cost = 0.02187795
Epoch: 5 accuracy = 0.12786026
Epoch: 6 cost = 0.02087924
Epoch: 6 accuracy = 0.12786026
Epoch: 7 cost = 0.01993689
Epoch: 7 accuracy = 0.12786026
Epoch: 8 cost = 0.01906220
Epoch: 8 accuracy = 0.12786026
Epoch: 9 cost = 0.01826031
Epoch: 9 accuracy = 0.12786028
Epoch: 10 cost = 0.01752586
Epoch: 10 accuracy = 0.12786026
Epoch: 11 cost = 0.01684174
Epoch: 11 accuracy = 0.12786026
Epoch: 12 cost = 0.01618174
Epoch: 12 accuracy = 0.12786026
Epoch: 13 cost = 0.01551575
Epoch: 13 accuracy = 0.12786026
Epoch: 14 cost = 0.01481585
Epoch: 14 accuracy = 0.12786026
Epoch: 15 cost = 0.01406221
Epoch: 15 accuracy = 0.12786026
Epoch: 16 cost = 0.01324943
Epoch: 16 accuracy = 0.12786026
Epoch: 17 cost = 0.01239465
Epoch: 17 accuracy = 0.1278602

Epoch: 143 cost = 0.00568222
Epoch: 143 accuracy = 0.49445772
Epoch: 144 cost = 0.00565369
Epoch: 144 accuracy = 0.55761254
Epoch: 145 cost = 0.00544943
Epoch: 145 accuracy = 0.62262321
Epoch: 146 cost = 0.00536575
Epoch: 146 accuracy = 0.63884425
Epoch: 147 cost = 0.00531886
Epoch: 147 accuracy = 0.64563262
Epoch: 148 cost = 0.00528118
Epoch: 148 accuracy = 0.64860731
Epoch: 149 cost = 0.00524615
Epoch: 149 accuracy = 0.65112424
Epoch: 150 cost = 0.00521191
Epoch: 150 accuracy = 0.65397185
Epoch: 151 cost = 0.00518321
Epoch: 151 accuracy = 0.66020089
Epoch: 152 cost = 0.00527054
Epoch: 152 accuracy = 0.66210771
Epoch: 153 cost = 0.00548931
Epoch: 153 accuracy = 0.66233659
Epoch: 154 cost = 0.00513675
Epoch: 154 accuracy = 0.66226023
Epoch: 155 cost = 0.00504772
Epoch: 155 accuracy = 0.66259080
Epoch: 156 cost = 0.00501771
Epoch: 156 accuracy = 0.66396374
Epoch: 157 cost = 0.00499898
Epoch: 157 accuracy = 0.66970968
Epoch: 158 cost = 0.00499091
Epoch: 158 accuracy = 0.69732088
Epoch: 1

Epoch: 285 accuracy = 0.76385713
Epoch: 286 cost = 0.00333960
Epoch: 286 accuracy = 0.76276374
Epoch: 287 cost = 0.00334862
Epoch: 287 accuracy = 0.76357734
Epoch: 288 cost = 0.00333264
Epoch: 288 accuracy = 0.76461977
Epoch: 289 cost = 0.00334090
Epoch: 289 accuracy = 0.76271296
Epoch: 290 cost = 0.00339114
Epoch: 290 accuracy = 0.76268762
Epoch: 291 cost = 0.00334486
Epoch: 291 accuracy = 0.76828092
Epoch: 292 cost = 0.00325874
Epoch: 292 accuracy = 0.76868778
Epoch: 293 cost = 0.00323848
Epoch: 293 accuracy = 0.76489955
Epoch: 294 cost = 0.00328197
Epoch: 294 accuracy = 0.75856882
Epoch: 295 cost = 0.00331274
Epoch: 295 accuracy = 0.76052654
Epoch: 296 cost = 0.00325556
Epoch: 296 accuracy = 0.76477242
Epoch: 297 cost = 0.00320277
Epoch: 297 accuracy = 0.76761997
Epoch: 298 cost = 0.00317546
Epoch: 298 accuracy = 0.77145898
Epoch: 299 cost = 0.00318340
Epoch: 299 accuracy = 0.77003533
Epoch: 300 cost = 0.00336016
Epoch: 300 accuracy = 0.75925523
Epoch: 301 cost = 0.00340661
Epoch: 3

In [19]:
# on 200 epoch

print(sess.run([accuracy], feed_dict={X: X_test, Y: y_test11}))

[0.68830943]


In [28]:
# on 400 epoch

print(sess.run([accuracy], feed_dict={X: X_test, Y: y_test11}))

[0.46640238]
